# Imports

In [ ]:
import utils.data_prc as dp 
import utils.build_network as bn
import utils.compilation_opt as cpo

In [ ]:
dp.dataset_pickle("fer2013.csv", False)

In [ ]:
X_train, y_train, X_test, y_test = dp.dataset_loading("fer2013")

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
emotion = { 0:'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy',
           4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [ ]:
num_classes = len(emotion)
samples_per_class = 7

for y, cls in emotion.items():
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=True)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(reshape(X_train[idx], (48, 48)).astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)

print(y_train.shape)            
plt.show()


In [ ]:
y_train, y_test = bn.get_metadata(X_train, X_test, y_train, y_test)
print(y_train.shape)

In [ ]:
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

X_train = np.reshape(X_train,(X_train.shape[0], 48, 48,1))
X_test = np.reshape(X_test,(X_test.shape[0], 48, 48,1))
model = bn.build_model(X_train)
#plot_model(model, to_file='model.png')

SVG(model_to_dot(model).create(prog='dot', format='svg'))


In [ ]:
#opt = cpo.sgdOpt()
opt = cpo.rmsPropOpt()


model = cpo.compiling(model, opt)
model.summary()

In [ ]:
epochs = 20
batch_size = 128

print (X_train.shape, y_train.shape)
model, history = cpo.training(model, batch_size, epochs, X_train, y_train)

In [ ]:
cpo.eval_plot(model, X_train, y_train, history, epochs)

In [ ]:
cpo.eval_plot(model, X_test, y_test, history, epochs)